In [2]:
!pip install hnswlib

     |████████████████████████████████| 188 kB 5.4 MB/s eta 0:00:01
  ERROR: Command errored out with exit status 1:
   command: /Users/imutayuji/opt/anaconda3/envs/brave/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/zq/sbn3ykjs75q0rsf9hhc_318h0000gn/T/pip-install-3xjtv7hl/hnswlib/setup.py'"'"'; __file__='"'"'/private/var/folders/zq/sbn3ykjs75q0rsf9hhc_318h0000gn/T/pip-install-3xjtv7hl/hnswlib/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/zq/sbn3ykjs75q0rsf9hhc_318h0000gn/T/pip-wheel-iuqlfujt
       cwd: /private/var/folders/zq/sbn3ykjs75q0rsf9hhc_318h0000gn/T/pip-install-3xjtv7hl/hnswlib/
  Complete output (41 lines):
  running bdist_wheel
  running build
  running build_ext
  creating var
  creating var/folders
  creating var/folders/zq
  creating var/folders/zq/sbn3ykjs75q0rs

In [7]:
!pip install fasttext

     |████████████████████████████████| 68 kB 2.8 MB/s eta 0:00:01
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-macosx_10_15_x86_64.whl size=328428 sha256=e25e4d8cb890b43560d75caaf70204a799d4270725596b5434eaa08b4466d987
  Stored in directory: /Users/imutayuji/Library/Caches/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [8]:
import hnswlib
import numpy as np
import json
import gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot 
import fasttext.util

#英語のトレーニング済みモデル
fasttext.util.download_model("en", if_exists = "ignore")
ft = fasttext.load_model("cc.en.300.bin")

from datetime import datetime
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors

 (100.00%) [==================================================>]================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ] ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> 

In [5]:
#hnswグラフを作成するために、必要な関数

def fit_hnsw_index(features, ef = 100, M = 16, save_index_file = False):
    #HNSWグラフを作成するために便利な関数
    #features : 埋め込みを含むリストのリスト
    #ef, M: HNSWアルゴリズム を調整するためのパラメーター
    
    num_elements = len(features)
    labels_index = np.arange(num_elements)
    
    EMBEDDING_SIZE = len(features[0])
    
    #インデックスの宣言
    #可能なスペースオプションはl2, コサイン, ip
    p = hnswlib.Index(space = "l2", dim = EMBEDDING_SIZE)
    
    #初期化インデックス　：　要素の最大数を知る必要がある
    p.init_index(max_elements = num_elements, 
                          ef_construction = ef,
                          M = M)
    
    #要素の挿入
    int_labels = p.add_items(features, labels_index)
    
    p.set_ef(ef)
    
    if save_index_file:
        p.save_index(save_index_file)
        
    return p

In [6]:
#使い方
#ann_neighbor_indeces, ann_distance = p.knn_query(features, k)

NameError: name 'p' is not defined

In [13]:
path = "/Users/imutayuji/Downloads/"
data = []
with gzip.open(path + "meta_Cell_Phones_and_Accessories.json.gz") as f:
    for l in f:
        data.append(json.loads(l.strip()))
        
df = pd.DataFrame.from_dict(data)
df.fillna("", inplace = True)

df = df[~df.title.str.contains("getTime")]


df = df[df["main_cat"] == "Cell Phones & Accessories"]


#reset index
df.reset_index(inplace = True, drop = True)

# Only keep the title columns
df = df[["title"]]

print(df.shape)
df.head()

(527543, 1)


,title
0,Puppies Faceplate Hard Case Protector for Net1...
1,White Wolf Faceplate Protector Hard Case for S...
2,Camo Duck Grass Rubberized Hard Case Phone Fac...
3,Camoflague Camo Usa Deer Combo Hybrid Hard Cas...
4,Motorola H700 Black - Non-Retail Packaging


In [14]:
df["emb"] = df["title"].apply(ft.get_sentence_vector)

#埋め込み列をNN
X = [item.tolist() for item in df["emb"].values]

In [18]:
n_products = [1000, 10000, len(X)]

In [16]:
n_neighbors = [10, 100]

In [17]:
metrics = {'products':[], 'k':[], 'knn_time':[],'ann_time':[], 'pct_overlap':[]}

In [20]:
for products in tqdm(n_products):
    features = X[:products]
    
    for k in tqdm(n_neighbors):
        ann_start = datetime.now()
        p = fit_hnsw_index(features, ef=k*10)
        ann_neighbor_indices, ann_distances = p.knn_query(features, k)
        ann_end = datetime.now()
        
        metrics['ann_time'].append((ann_end - ann_start).total_seconds())

100%|██████████| 3/3 [14:26<00:00, 288.93s/it]


In [24]:
metrics["ann_time"]

[0.056296, 0.134813, 0.627234, 3.372761, 79.220481, 783.338619]